In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [ ]:
# filename = "wonderland.txt"
# raw_text = open(filename, 'r', encoding='utf-8').read()
# raw_text = raw_text.lower()
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text.lower()


# Load the data
file_path = 'wiki.train.raw'
text_data = load_data(file_path)

# # create mapping of unique chars to integers
# chars = sorted(list(set(raw_text)))
# char_to_int = dict((c, i) for i, c in enumerate(chars))

# # summarize the loaded data
# n_chars = len(raw_text)
# n_vocab = len(chars)
# print("Total Characters: ", n_chars)
# print("Total Vocab: ", n_vocab)

In [ ]:
print(len(text_data[0:30]))

30


In [ ]:
char_list= sorted(list(set(text_data)))      # 940 unique characters
num_characters= len(char_list)               # 940
total_characters= len(text_data)             # 10918893
char_to_int= dict((c, i) for i, c in enumerate(char_list))
print("#"*200)
print("the total number of characters in dataset ", len(text_data))
print("the length of vocab(unique characters) in dataset ", len(char_list))

########################################################################################################################################################################################################
the total number of characters in dataset  10918893
the length of vocab(unique characters) in dataset  940


In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
count=0
for i in range(0, num_characters - seq_length, 1):
    count+=i
    seq_in = text_data[i:i + seq_length]
    seq_out = text_data[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    if i< 5:
      print(dataX)
      print("the length of dataX is:", len(dataX))
      print(dataY)
      print("the length of dataY is:", len(dataY))
    dataY.append(char_to_int[seq_out])

print(f"This loops runs {count} number of times")
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

[[1, 0, 1, 30, 1, 61, 40, 51, 50, 64, 57, 48, 40, 1, 42, 47, 57, 54, 53, 48, 42, 51, 44, 58, 1, 48, 48, 48, 1, 30, 1, 0, 1, 0, 1, 58, 44, 53, 49, 134, 1, 53, 54, 1, 61, 40, 51, 50, 64, 57, 48, 40, 1, 20, 1, 27, 1, 60, 53, 57, 44, 42, 54, 57, 43, 44, 43, 1, 42, 47, 57, 54, 53, 48, 42, 51, 44, 58, 1, 9, 1, 49, 40, 55, 40, 53, 44, 58, 44, 1, 27, 1, 737, 685, 544, 620, 564, 616, 575, 612]]
the length of dataX is: 1
[]
the length of dataY is: 0
[[1, 0, 1, 30, 1, 61, 40, 51, 50, 64, 57, 48, 40, 1, 42, 47, 57, 54, 53, 48, 42, 51, 44, 58, 1, 48, 48, 48, 1, 30, 1, 0, 1, 0, 1, 58, 44, 53, 49, 134, 1, 53, 54, 1, 61, 40, 51, 50, 64, 57, 48, 40, 1, 20, 1, 27, 1, 60, 53, 57, 44, 42, 54, 57, 43, 44, 43, 1, 42, 47, 57, 54, 53, 48, 42, 51, 44, 58, 1, 9, 1, 49, 40, 55, 40, 53, 44, 58, 44, 1, 27, 1, 737, 685, 544, 620, 564, 616, 575, 612], [0, 1, 30, 1, 61, 40, 51, 50, 64, 57, 48, 40, 1, 42, 47, 57, 54, 53, 48, 42, 51, 44, 58, 1, 48, 48, 48, 1, 30, 1, 0, 1, 0, 1, 58, 44, 53, 49, 134, 1, 53, 54, 1, 61, 40

In [ ]:
# TEST CODE

# text_data = "hello world, how are you doing today? This is a sample text for explanation."
# seq_length= 50

# print(len(text_data))
# for i in range(0, len(text_data) - seq_length, 1):

#     seq_in = text_data[i:i + seq_length]
#     seq_out = text_data[i + seq_length]
#         # Print seq_in and seq_out for the first few iterations
#     # if i < 3:
#     print(f"Iteration {i + 1} - seq_in: '{seq_in}', seq_out: '{seq_out}'")

# OUTPUT::
# Iteration 1 - seq_in: 'hello world, how are you doing today? This is a sa', seq_out: 'm'
# Iteration 2 - seq_in: 'ello world, how are you doing today? This is a sam', seq_out: 'p'
# Iteration 3 - seq_in: 'llo world, how are you doing today? This is a samp', seq_out: 'l'



In [ ]:
# reshape X to be [samples, time steps, features]
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)

# normalizes the values in the tensor X by dividing each element by num_characters.
X = X / float(num_characters)
y = torch.tensor(dataY)
print(X.shape, y.shape)

torch.Size([840, 100, 1]) torch.Size([840])


In [ ]:
class CharacterLM(nn.Module):
  def __init__(self):
    super().__init__()
    # input_size=1. because the input is character-level data

    self.lstm= nn.LSTM(input_size=1, hidden_size= 256, num_layers=1, batch_first=True)
    self.dropout= nn.Dropout(0.2)
    self.linear= nn.Linear(256, num_characters)

  def forward(self, x):
    x, _= self.lstm(x)
    # -1 take the output corresponding to the last time step of each sequence in the batch.
    #taking only last output
    x= x[:, -1, :]
    # producing output
    x= self.linear(self.dropout(x))
    return x



In [ ]:
n_epochs= 10
batch_size=128
# intialize CharacterLM model
model= CharacterLM()

optimizer= optim.Adam(model.parameters())
loss_function= nn.CrossEntropyLoss()
loader= data.DataLoader(data.TensorDataset(X, y), shuffle=True, batch_size=batch_size)

best_model= None
best_loss= 1000

for epoch in range(n_epochs):
  model.train()
  for X_batch, y_batch in loader:
    y_pred= model(X_batch)
    loss= loss_function(y_pred, y_batch)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # validation
  model.eval()
  loss= 0
  with torch.no_grad():
    for X_batch, y_batch in loader:
      y_pred= model(X_batch)
      loss+= loss_function(y_pred, y_batch)

    if loss < best_loss:
      best_loss= loss
      best_model= model.state_dict()
    print("Epoch %d: Crossentropy loss: %.4f" % (epoch,loss))

torch.save([best_model, char_to_int], "single-character-model.pth")

Epoch 0: Crossentropy loss: 46.4051
Epoch 1: Crossentropy loss: 24.8039
Epoch 2: Crossentropy loss: 21.3184
Epoch 3: Crossentropy loss: 21.0173
Epoch 4: Crossentropy loss: 20.7599
Epoch 5: Crossentropy loss: 20.8311
Epoch 6: Crossentropy loss: 20.9289
Epoch 7: Crossentropy loss: 20.9080
Epoch 8: Crossentropy loss: 20.7282
Epoch 9: Crossentropy loss: 20.7868


In [ ]:
char_to_int= dict((c, i) for i, c in enumerate(char_list))

In [ ]:
import numpy as np

best_model, char_to_int= torch.load('single-character-model.pth')
int_to_char= dict((i, c) for c, i in char_to_int.items())

# reloading the saved model
class CharacterLM(nn.Module):
  def __init__(self):
    super().__init__()
    self.lstm= nn.LSTM(input_size=1, hidden_size=256, num_layers=1, batch_first=True)
    self.Dropout= nn.Dropout(0.2)
    self.linear= nn.Linear(256, num_characters)

  def forward(self, x):
    x, _= self.lstm(x)

    # takes only the last output:
    x= x[:, -1, :]
    x= self.Linear(self.dropout(x))
    return x

lmodel= CharacterLM()
lmodel.load_state_dict(best_model)

prompt= "i want to play the "
pattern= [char_to_int[c] for c in prompt]

model.eval()
with torch.no_grad():
  for i in range(1000):
    x= np.reshape(pattern, (1, len(pattern), 1)) /float(num_characters)
    x= torch.tensor(x, dtype=torch.float32)

    # generate logits as output from the model
    prediction= model(x)

    # convert logits into one character
    index= int(prediction.argmax())
    result= int_to_char[index]

    print(result, end="")

    # append the new character into the prompt for the next iteration
    pattern.append(index)
    pattern= pattern[1:]

print("\nFinished!!!!!!!!!!!!!!!!!")

In [ ]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text.lower()


# Load the data
file_path = 'wiki.train.raw'
text_data = load_data(file_path)

In [ ]:
from tqdm import tqdm

def load_data(filename, seq_length):
    raw_text = open(filename, 'r', encoding='utf-8').read().lower()
    chars = sorted(list(set(raw_text)))
    char_to_int = {c: i for i, c in enumerate(chars)}
    dataX, dataY = [], []
    for i in range(0, len(raw_text) - seq_length):
        seq_in = raw_text[i:i + seq_length]
        seq_out = raw_text[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])
    return dataX, dataY, char_to_int, len(chars), raw_text

class CharModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate):
        super(CharModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_rate)
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.linear(self.dropout(x))
        return x


def train_model(model, data_loader, optimizer, loss_fn, n_epochs, checkpoint_path="model_checkpoint.pth"):
    """
    Trains the model and saves checkpoints every 5 epochs.

    Parameters:
    - model: The PyTorch model to train.
    - data_loader: DataLoader for the training data.
    - optimizer: Optimizer for the model's parameters.
    - loss_fn: Loss function to use for training.
    - n_epochs: Number of epochs to train for.
    - checkpoint_path: Base path for saving model checkpoints.

    Returns:
    - The state dictionary of the best model.
    """
    best_model_wts = None
    best_loss = np.inf
    total_batches = len(data_loader)

    for epoch in range(1, n_epochs + 1):
        model.train()
        running_loss = 0.0
        batch_progress = tqdm(enumerate(data_loader, 1), total=total_batches, desc=f"Epoch {epoch}/{n_epochs}")

        for batch_idx, (X_batch, y_batch) in batch_progress:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            batch_progress.set_postfix({'Batch Loss': loss.item()})

        epoch_loss = running_loss / total_batches
        print(f'\nEpoch {epoch} Average Loss: {epoch_loss:.4f}')

        # Update best model if current epoch loss is lower
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            best_model_wts = model.state_dict()
            print(f"New best model found at epoch {epoch} with loss {epoch_loss:.4f}")

        # Save checkpoint every 5 epochs
        if epoch % 5 == 0:
            checkpoint_filename = f"{checkpoint_path}_epoch_{epoch}.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': epoch_loss,
            }, checkpoint_filename)
            print(f"Checkpoint saved at '{checkpoint_filename}'")

    return best_model_wts

def generate_text(model, start_prompt, char_to_int, int_to_char, n_vocab, generate_length=1000):
    model.eval()
    pattern = [char_to_int[c] for c in start_prompt.lower()]
    print('Prompt: "%s"' % start_prompt)
    with torch.no_grad():
        for i in range(generate_length):
            x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
            x = torch.tensor(x, dtype=torch.float32)
            prediction = model(x)
            index = prediction.argmax().item()
            print(int_to_char[index], end="")
            pattern.append(index)
            pattern = pattern[1:]
    print("\nDone.")


In [ ]:
torch.cuda.empty_cache()

In [ ]:
import numpy as np

In [ ]:
def main():
    filename = "wonderland.txt"
    seq_length = 100
    # Assuming load_data returns raw_text as the last item for prompt generation
    dataX, dataY, char_to_int, n_vocab, raw_text = load_data(filename, seq_length)

    X = torch.tensor(dataX, dtype=torch.float32).reshape(-1, seq_length, 1) / float(n_vocab)
    y = torch.tensor(dataY)
    data_loader = data.DataLoader(data.TensorDataset(X, y), batch_size=512, shuffle=True)

    model = CharModel(1, 256, 10, n_vocab, 0.2)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()

    # Specify the directory where you want to save the checkpoints
    checkpoint_dir = "/content/model_checkpoint"
    best_model_wts = train_model(model, data_loader, optimizer, loss_fn, 50, checkpoint_dir)

    # Save the best model weights and char_to_int mapping
    torch.save([best_model_wts, char_to_int], "complex_single-char.pth")

if __name__ == "__main__":
    main()


Epoch 1/50 Train Loss: 3.1397 Val Loss: 2.9849


Epoch 2/50 Train Loss: 2.9477 Val Loss: 2.9777


Epoch 3/50 Train Loss: 2.9534 Val Loss: 2.9764


Epoch 4/50 Train Loss: 2.9457 Val Loss: 2.9776


Epoch 5/50 Train Loss: 2.9339 Val Loss: 2.9779
Checkpoint saved at '/content/model_checkpoint_epoch_5.pth'


...

Epoch 22/50 Train Loss: 2.9327 Val Loss: 2.9770


Epoch 23/50 Train Loss: 2.9338 Val Loss: 2.9740


Epoch 24/50 Train Loss: 2.9292 Val Loss: 2.9728


Epoch 25/50 Train Loss: 2.9377 Val Loss: 2.9734
Checkpoint saved at '/content/model_checkpoint_epoch_25.pth'


Epoch 26/50 [Train]:  77%|███████▋  | 17/22 [05:08<01:31, 18.26s/it, Batch Loss=2.89]

In [ ]:
import numpy as np
import torch

def inference(model, start_prompt, char_to_int, int_to_char, n_vocab, generate_length=1000):
    """
    Generate text from a trained character-level RNN model.

    Parameters:
    - model: The trained PyTorch model for character-level text generation.
    - start_prompt: The starting text prompt for text generation.
    - char_to_int: Dictionary mapping characters to integers.
    - int_to_char: Dictionary mapping integers back to characters.
    - n_vocab: The size of the vocabulary (number of unique characters).
    - generate_length: Optional; Number of characters to generate.

    Returns:
    - Generated text as a string.
    """

    # Ensure model is in evaluation mode
    model.eval()

    # Initialize the pattern from the start prompt
    pattern = [char_to_int[c] for c in start_prompt.lower()]

    # Store the generated text
    generated_text = start_prompt

    with torch.no_grad():
        for i in range(generate_length):
            # Prepare the input tensor
            x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
            x = torch.tensor(x, dtype=torch.float32)

            # Generate logits
            prediction = model(x)

            # Convert logits to next character index and update the pattern
            index = prediction.argmax().item()
            next_char = int_to_char[index]
            generated_text += next_char

            pattern.append(index)
            pattern = pattern[1:]  # Move the window for the next character

    return generated_text

# Assuming model, char_to_int, int_to_char, and n_vocab are already defined and loaded

start_prompt = "The adventures of "
generated_text = inference(model, start_prompt, char_to_int, int_to_char, n_vocab, generate_length=200)

print("Generated Text:")
print(generated_text)

In [ ]:
def evaluate_model(model, test_loader, loss_fn):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_characters = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            loss = loss_fn(output, y_batch)
            total_loss += loss.item()

            _, predicted = torch.max(output, dim=1)
            correct_predictions += (predicted == y_batch).sum().item()
            total_characters += y_batch.size(0)

    avg_loss = total_loss / len(test_loader.dataset)
    accuracy = correct_predictions / total_characters
    perplexity = np.exp(avg_loss)

    return avg_loss, accuracy, perplexity


filename = "wonderland.txt"
seq_length = 100
dataX, dataY, char_to_int, n_vocab, raw_text = load_data(filename, seq_length)

X = torch.tensor(dataX, dtype=torch.float32).reshape(-1, seq_length, 1) / float(n_vocab)
y = torch.tensor(dataY)
test_loader = data.DataLoader(data.TensorDataset(X, y), batch_size=128, shuffle=True)
# Assuming you have a DataLoader for your test dataset
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
loss_fn = nn.CrossEntropyLoss()

avg_loss, accuracy, perplexity = evaluate_model(model, test_loader, loss_fn)
print(f"Loss: {avg_loss}, Accuracy: {accuracy}, Perplexity: {perplexity}")


NameError: name 'model' is not defined